# 神经网络解常微分方程例子

In [32]:
from ode_example.model import *
from ode_example.data import *
from ode_example.criterion import *
from ode_example.train import *

In [35]:
%pylab
%matplotlib qt5

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [36]:
model = ApproxSolve(15, 2) # 10 layer 2 node
# model = torch.load("model.pth")
data_sampler = DataSampler(100, 1)  # 100 data, 10 boundary data
criterion = Criterion(model, data_sampler)

train = Train(criterion)
train.train(5000, 1e-4)

torch.save(model, 'model.pth')


Step 49 - lr 0.0001 -  loss: 232.63445220947267
Step 99 - lr 0.0001 -  loss: 227.4049853515625
Step 149 - lr 0.0001 -  loss: 223.38107330322265
Step 199 - lr 0.0001 -  loss: 219.47606689453124
Step 249 - lr 0.0001 -  loss: 215.21388610839844
Step 299 - lr 0.0001 -  loss: 211.1217874145508
Step 349 - lr 0.0001 -  loss: 207.35233276367188
Step 399 - lr 0.0001 -  loss: 203.07132415771486
Step 449 - lr 0.0001 -  loss: 199.7158172607422
Step 499 - lr 0.0001 -  loss: 195.8965921020508
Step 549 - lr 0.0001 -  loss: 192.22187896728516
Step 599 - lr 0.0001 -  loss: 188.68462341308594
Step 649 - lr 0.0001 -  loss: 185.2532928466797
Step 699 - lr 0.0001 -  loss: 181.96983978271484
Step 749 - lr 0.0001 -  loss: 178.5954150390625
Step 799 - lr 0.0001 -  loss: 175.53582336425782
Step 849 - lr 0.0001 -  loss: 172.32372131347657
Step 899 - lr 0.0001 -  loss: 169.27655853271483
Step 949 - lr 0.0001 -  loss: 166.34354370117188
Step 999 - lr 0.0001 -  loss: 163.23002288818358
Step 1049 - lr 0.0001 -  los

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
